In [ ]:
1. 학습,검증,테스트 데이터 7:1:2로 분할
2. MLP로 사용자-식당 간의 비선형적 패턴 학습(64차원)
3. ABSA결과를 ReviewAspectModule에 입력해 MLP로 학습(32차원)
4. 2번과 3번을 결합(concatenation)해 96차원 벡터 만듬
5. 4번을 prediction_mlp에 넣어 별점 예측

##### import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
import os # 모델 저장을 위해 os 모듈 임포트

# JSONL 파일 로드 (lines=True 필수)
# 파일 경로를 실제 파일 경로로 바꿔주세요.
df = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출 (여기서 명시적으로 .copy()를 사용하여 새로운 DataFrame을 만듭니다)
df_processed = df[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# user_id와 business_id를 연속적인 정수 ID로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

# .loc를 사용하여 값을 할당하여 경고를 방지합니다.
df_processed.loc[:, 'user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed.loc[:, 'business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)

# 1. 먼저 학습+검증 세트와 테스트 세트로 8:2 비율로 분할
# (7+1):2 = 8:2
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)

# 2. 학습+검증 세트를 다시 학습 세트와 검증 세트로 7:1 비율로 분할
# val_size_ratio는 train_val_df의 1/8이므로, 전체 데이터의 1/10이 됩니다.
val_size_ratio = 1 / 8
train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=42)

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")

# --- PyTorch Dataset 및 DataLoader 정의 ---
class ReviewDataset(Dataset):
    def __init__(self, df):
        self.user_ids = torch.tensor(df['user_encoded'].values, dtype=torch.long)
        self.business_ids = torch.tensor(df['business_encoded'].values, dtype=torch.long)
        self.sentiment_vectors = torch.tensor(np.array(df['sentiment_vector'].tolist()), dtype=torch.float)
        self.stars = torch.tensor(df['stars'].values, dtype=torch.float)

    def __len__(self):
        return len(self.stars)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.business_ids[idx], self.sentiment_vectors[idx], self.stars[idx]

# --- 모델 아키텍처 정의 (이전과 동일) ---
class CustomerRestaurantInteractionModule(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim, mlp_dims):
        super(CustomerRestaurantInteractionModule, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        layers = []
        input_dim = embedding_dim * 2
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = mlp_dims[-1] if mlp_dims else embedding_dim * 2

    def forward(self, user_ids, business_ids):
        user_vec = self.user_embedding(user_ids)
        business_vec = self.business_embedding(business_ids)
        combined_vec = torch.cat((user_vec, business_vec), dim=1)
        interaction_features = self.mlp(combined_vec)
        return interaction_features

class ReviewAspectModule(nn.Module):
    def __init__(self, sentiment_vector_dim, mlp_dims):
        super(ReviewAspectModule, self).__init__()
        layers = []
        input_dim = sentiment_vector_dim
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = mlp_dims[-1] if mlp_dims else sentiment_vector_dim

    def forward(self, sentiment_vectors):
        aspect_features = self.mlp(sentiment_vectors)
        return aspect_features

class AATRec(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim,
                 user_biz_mlp_dims, aspect_mlp_dims, final_mlp_dims,
                 sentiment_vector_dim=15):
        super(AATRec, self).__init__()
        self.customer_restaurant_interaction_module = CustomerRestaurantInteractionModule(
            num_users, num_businesses, embedding_dim, user_biz_mlp_dims
        )
        self.review_aspect_module = ReviewAspectModule(
            sentiment_vector_dim, aspect_mlp_dims
        )
        final_input_dim = self.customer_restaurant_interaction_module.output_dim + \
                          self.review_aspect_module.output_dim
        layers = []
        input_dim = final_input_dim
        for dim in final_mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        layers.append(nn.Linear(input_dim, 1))
        self.prediction_mlp = nn.Sequential(*layers)

    def forward(self, user_ids, business_ids, sentiment_vectors):
        user_biz_features = self.customer_restaurant_interaction_module(user_ids, business_ids)
        aspect_features = self.review_aspect_module(sentiment_vectors)
        combined_features = torch.cat((user_biz_features, aspect_features), dim=1)
        predicted_rating = self.prediction_mlp(combined_features)
        return predicted_rating.squeeze()

# --- 하이퍼파라미터 설정 ---
embedding_dim = 64
user_biz_mlp_dims = [128, 64]
aspect_mlp_dims = [64, 32]
final_mlp_dims = [64, 32]
learning_rate = 0.001
epochs = 50 # 조기 종료를 고려하여 충분히 큰 에폭 설정
batch_size = 256

# --- 조기 종료(Early Stopping) 설정 ---
patience = 5 # 검증 RMSE가 개선되지 않아도 기다릴 에폭 수
min_delta = 0.001 # 검증 RMSE가 개선되었다고 판단할 최소 변화량
best_val_rmse = float('inf') # 초기 최적 검증 RMSE (무한대로 설정)
epochs_no_improve = 0 # 검증 RMSE가 개선되지 않은 에폭 수 카운터
model_path = 'best_aatrec_model.pt' # 최적 모델 저장 경로

# 데이터셋 및 DataLoader 생성
train_dataset = ReviewDataset(train_df)
val_dataset = ReviewDataset(val_df)
test_dataset = ReviewDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 인스턴스 생성
model = AATRec(num_users, num_businesses, embedding_dim,
               user_biz_mlp_dims, aspect_mlp_dims, final_mlp_dims,
               sentiment_vector_dim=15)

# 손실 함수 및 옵티마이저
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# --- 학습 루프 (조기 종료 추가) ---
print("모델 학습 시작...")
for epoch in range(epochs):
    # --- 학습 단계 ---
    model.train()
    total_train_loss = 0
    for user_ids, business_ids, sentiment_vectors, stars in train_loader:
        optimizer.zero_grad()
        predictions = model(user_ids, business_ids, sentiment_vectors)
        loss = criterion(predictions, stars)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # --- 검증 단계 ---
    model.eval()
    total_val_loss = 0
    val_predictions = []
    val_true_ratings = []
    with torch.no_grad():
        for user_ids, business_ids, sentiment_vectors, stars in val_loader:
            predictions = model(user_ids, business_ids, sentiment_vectors)
            loss = criterion(predictions, stars)
            total_val_loss += loss.item()
            val_predictions.extend(predictions.tolist())
            val_true_ratings.extend(stars.tolist())

    current_val_rmse = np.sqrt(mean_squared_error(val_true_ratings, val_predictions))

    print(f"Epoch {epoch+1}/{epochs}, "
          f"Train Loss: {total_train_loss / len(train_loader):.4f}, "
          f"Val Loss: {total_val_loss / len(val_loader):.4f}, "
          f"Val RMSE: {current_val_rmse:.4f}")

    # --- 조기 종료 로직 ---
    if current_val_rmse < best_val_rmse - min_delta: # 검증 RMSE가 개선되었다면
        best_val_rmse = current_val_rmse
        epochs_no_improve = 0 # 개선이 있었으므로 카운트 초기화
        torch.save(model.state_dict(), model_path) # 최적 모델 가중치 저장
        print(f"  --> 검증 RMSE 개선됨. 최적 모델 저장됨: {best_val_rmse:.4f}")
    else: # 개선되지 않았다면
        epochs_no_improve += 1 # 카운트 증가
        print(f"  --> 검증 RMSE 개선되지 않음. 대기 중... ({epochs_no_improve}/{patience})")
        if epochs_no_improve == patience:
            print(f"조기 종료! {patience} 에폭 동안 검증 RMSE 개선이 없었습니다.")
            break # 학습 루프 종료

print("모델 학습 완료.")

# --- 최종 테스트 및 RMSE 계산 ---
# 최적의 모델 가중치를 로드하여 테스트
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print(f"최적의 모델 가중치 '{model_path}' 로드 완료.")
else:
    print(f"최적의 모델 가중치 '{model_path}'를 찾을 수 없습니다. 현재 모델 상태로 테스트를 진행합니다.")

model.eval()
test_predictions = []
true_ratings = []

with torch.no_grad():
    for user_ids, business_ids, sentiment_vectors, stars in test_loader:
        predictions = model(user_ids, business_ids, sentiment_vectors)
        test_predictions.extend(predictions.tolist())
        true_ratings.extend(stars.tolist())

rmse = np.sqrt(mean_squared_error(true_ratings, test_predictions))
print(f"\n최종 테스트 RMSE (최적 모델 기준): {rmse:.4f}")

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import train_test_split as surprise_train_test_split
from surprise import accuracy # RMSE 계산을 위함

# JSONL 파일 로드
# 파일 경로를 실제 파일 경로로 바꿔주세요.
df = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출 및 SettingWithCopyWarning 해결
df_processed = df[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# user_id와 business_id를 연속적인 정수 ID로 인코딩 (AAT-Rec과의 일관성을 위해 유지)
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df_processed.loc[:, 'user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed.loc[:, 'business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

# 데이터 분할 (7:1:2 비율)
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
val_size_ratio = 1 / 8
train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=42)

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")

# UBCF 모델 구현 및 평가
# Surprise 라이브러리를 위한 데이터 로드
reader = Reader(rating_scale=(1, 5))

# 학습 데이터 (train_df)로 train_set 구성
train_set_for_surprise = train_df[['user_id', 'business_id', 'stars']].copy()
data_for_train = Dataset.load_from_df(train_set_for_surprise, reader)
train_set = data_for_train.build_full_trainset()

# 테스트 데이터 (test_df)로 test_set 구성
test_set = [(row['user_id'], row['business_id'], row['stars']) for _, row in test_df.iterrows()]

# UBCF (User-Based Collaborative Filtering) 모델 정의
ubcf_model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True}, k=100)

print("\nUBCF 모델 학습 시작...")
ubcf_model.fit(train_set)
print("UBCF 모델 학습 완료.")

print("UBCF 모델 예측 시작...")
predictions_ubcf = ubcf_model.test(test_set)
print("UBCF 모델 예측 완료.")

rmse_ubcf = accuracy.rmse(predictions_ubcf, verbose=False)
print(f"\nUBCF (n=100) 테스트 RMSE: {rmse_ubcf:.4f}")


# IBCF (Item-Based Collaborative Filtering) 구현 및 평가
ibcf_model = KNNBasic(sim_options={'name': 'cosine', 'user_based': False}, k=100)

print("\nIBCF 모델 학습 시작...")
ibcf_model.fit(train_set) # UBCF와 동일한 train_set 사용
print("IBCF 모델 학습 완료.")

print("IBCF 모델 예측 시작...")
predictions_ibcf = ibcf_model.test(test_set) # UBCF와 동일한 test_set 사용
print("IBCF 모델 예측 완료.")

rmse_ibcf = accuracy.rmse(predictions_ibcf, verbose=False)
print(f"\nIBCF (n=100) 테스트 RMSE: {rmse_ibcf:.4f}")

전체 데이터 수: 447796
학습 데이터 수: 313456 (70.00%)
검증 데이터 수: 44780 (10.00%)
테스트 데이터 수: 89560 (20.00%)

UBCF 모델 학습 시작...
Computing the cosine similarity matrix...
Done computing similarity matrix.
UBCF 모델 학습 완료.
UBCF 모델 예측 시작...
UBCF 모델 예측 완료.

UBCF (n=100) 테스트 RMSE: 1.1139

IBCF 모델 학습 시작...
Computing the cosine similarity matrix...
Done computing similarity matrix.
IBCF 모델 학습 완료.
IBCF 모델 예측 시작...
IBCF 모델 예측 완료.

IBCF (n=100) 테스트 RMSE: 1.1788
